In [45]:
import os
import pandas as pd

# Set the directory where the text files are located
songs_dir = "songs"
data = []
# Loop through each file in the directory
for root, dirs, files in os.walk(songs_dir):
    for file in files:
        if file.endswith(".txt"):
            artist = os.path.basename(root)
            with open(os.path.join(root, file), 'r', encoding="utf8") as f:
                lyrics = f.read().replace('\n', ' ')
                # Add the data to the DataFrame
                data.append([artist, lyrics])
                
# Create an empty DataFrame to store the data
df = pd.DataFrame(data, columns=['Artist', 'Lyrics'])
# Export the DataFrame to a CSV file
df.to_csv('lyrics.csv', index=False)


In [46]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Load the data from the CSV file
data = pd.read_csv("lyrics.csv")

nltk.download('stopwords')
nltk.download('punkt')

# Create a list of stopwords to remove
stop_words = set(stopwords.words("english"))
stop_words.add("verse")
stop_words.add("intro")

# Create a stemmer to use for word stemming
stemmer = PorterStemmer()

# Preprocess each lyric in the DataFrame
for i, row in data.iterrows():
#Convert the lyric to lowercase
    lyric = str(row["Lyrics"]).lower()
    
    split_lyric = lyric.split(" lyrics[", 1)
    
    # Check if the split was successful
    if len(split_lyric) > 1:
        # If successful, take the second part of the split
        cleaned_lyric = split_lyric[1]
    else:
        # If not successful, take the original lyric
        cleaned_lyric = lyric

    #Tokenize the lyric into words
    words = word_tokenize(cleaned_lyric)

    #Remove stop words and punctuation
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]

    #Stem each word
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    #Join the stemmed words back into a single string
    preprocessed_lyric = " ".join(stemmed_words)

    #Replace the original lyric with the preprocessed lyric in the DataFrame
    data.at[i, "Lyrics"] = preprocessed_lyric

# Export the preprocessed DataFrame to a CSV file
data.to_csv("preprocessed_lyrics.csv", index=False)



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jarraomar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jarraomar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

data = pd.read_csv("preprocessed_lyrics.csv")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["Lyrics"])
sequences = tokenizer.texts_to_sequences(data["Lyrics"])

In [95]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

artists = data["Artist"].unique()
# Encode the artist names as integer labels
label_encoder = LabelEncoder()
data["Artist"] = label_encoder.fit_transform(data["Artist"])

# Initialize empty dataframes for training, validation, and testing
train_df = pd.DataFrame(columns=["Artist", "Lyrics"])
val_df = pd.DataFrame(columns=["Artist", "Lyrics"])
test_df = pd.DataFrame(columns=["Artist", "Lyrics"])

for artist in artists:
    # Get the data for the current artist
    artist_data = data[data["Artist"] == artist]
    
    # Split the artist data into training, validation, and testing sets
    artist_train, artist_test = train_test_split(artist_data, test_size=0.2, random_state=42)
    artist_train, artist_val = train_test_split(artist_train, test_size=0.2, random_state=42)
    
    # Concatenate the artist training, validation, and testing dataframes with the overall training, validation, and testing dataframes
    train_df = pd.concat([train_df, artist_train])
    val_df = pd.concat([val_df, artist_val])
    test_df = pd.concat([test_df, artist_test])

# Create a directory to store the CSV files
directory = "data_splits"
if not os.path.exists(directory):
    os.makedirs(directory)

train_df.to_csv(os.path.join(directory, "train.csv"), index=False)
val_df.to_csv(os.path.join(directory, "val.csv"), index=False)
test_df.to_csv(os.path.join(directory, "test.csv"), index=False)


In [147]:
import pandas as pd
import numpy as np
import gensim
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

train = pd.read_csv('data_splits/train.csv')
val = pd.read_csv('data_splits/val.csv')
test = pd.read_csv('data_splits/test.csv')

#Train a Word2Vec model on the preprocessed lyrics data:
sentences = [row.split() for row in train['Lyrics']]
model = gensim.models.Word2Vec(sentences, min_count=1)

#Creation of MLP model:
model_mlp = Sequential()
model_mlp.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=100))
model_mlp.add(GlobalMaxPooling1D())
model_mlp.add(Dense(100, activation='relu'))
model_mlp.add(Dropout(0.5))
model_mlp.add(Dense(128, activation='relu'))
model_mlp.add(Dense(128, activation='relu'))
model_mlp.add(Dropout(0.5))
model_mlp.add(Dense(8, activation='softmax'))

tokenizer.fit_on_texts(train['Lyrics'])
sequences_train = tokenizer.texts_to_sequences(train['Lyrics'])
sequences_val = tokenizer.texts_to_sequences(val['Lyrics'])
sequences_test = tokenizer.texts_to_sequences(test['Lyrics'])
word_index = tokenizer.word_index
X_train = pad_sequences(sequences_train, maxlen=100)
X_val = pad_sequences(sequences_val, maxlen=100)
X_test = pad_sequences(sequences_test, maxlen=100)

# One-hot encode the target variable
y_train = to_categorical(train['Artist'])
y_val = to_categorical(val['Artist'])
y_test = to_categorical(test['Artist'])


In [148]:
es = EarlyStopping(monitor='val_loss', patience=2)
model_mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model)

Word2Vec<vocab=2623, vector_size=100, alpha=0.025>


In [149]:
model_mlp.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64, callbacks=[es])

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 2.0836 - accuracy: 0.0833 - val_loss: 2.0798 - val_accuracy: 0.1250
Epoch 2/50
1/1 [==============================] - 0s 36ms/step - loss: 2.0803 - accuracy: 0.1250 - val_loss: 2.0794 - val_accuracy: 0.1250
Epoch 3/50
1/1 [==============================] - 0s 51ms/step - loss: 2.0741 - accuracy: 0.1875 - val_loss: 2.0792 - val_accuracy: 0.1250
Epoch 4/50
1/1 [==============================] - 0s 55ms/step - loss: 2.0866 - accuracy: 0.1667 - val_loss: 2.0791 - val_accuracy: 0.1250
Epoch 5/50
1/1 [==============================] - 0s 52ms/step - loss: 2.0825 - accuracy: 0.1458 - val_loss: 2.0790 - val_accuracy: 0.1250
Epoch 6/50
1/1 [==============================] - 0s 51ms/step - loss: 2.0759 - accuracy: 0.1875 - val_loss: 2.0790 - val_accuracy: 0.1250
Epoch 7/50
1/1 [==============================] - 0s 51ms/step - loss: 2.0712 - accuracy: 0.0833 - val_loss: 2.0790 - val_accuracy: 0.1250
Epoch 8/50
1/1 [=============

In [150]:
test_loss, test_acc = model_mlp.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

1/1 - 0s - loss: 2.0728 - accuracy: 0.1875 - 31ms/epoch - 31ms/step
Test accuracy: 0.1875


In [146]:
from keras import backend as K
K.clear_session()
del model_mlp